In [32]:
import streamlit as st 
import pandas as pd
import datetime as dt
import numpy as np
import pandas_ta as ta

In [33]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///CryptoDB.db')

In [34]:
symbols = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', engine).name.to_list()

In [ ]:
st.title('Welcome to live TA platform')

In [35]:
def applytechnicals(df):
    df["EMA_5"] = ta.ema(df['c'], length=5)
    df["EMA_20"] = ta.ema(df['c'], length=20)
    df.dropna(inplace=True)

In [50]:
def qry(symbol):
    now = dt.datetime.utcnow()
    before = now - dt.timedelta(minutes=200)
    qry_str = f"""SELECT E,c FROM '{symbol}' WHERE E >= '{before}'"""
    df = pd.read_sql(qry_str,engine)
    df.E = pd.to_datetime(df.E)
    df = df.set_index('E')
    df = df.resample('5min').last() # 15 min charts
    applytechnicals(df)
    df['position'] = np.where(df["EMA_5"] > df["EMA_20"], 1, 2)
    return df

In [51]:
qry('BTCUSDT')

,c,EMA_5,EMA_20,position
E,,,,
2022-09-11 19:20:00,21592.62,21591.413639,21611.149000,2
2022-09-11 19:25:00,21589.63,21590.819093,21609.099571,2
2022-09-11 19:30:00,21573.78,21585.139395,21605.735803,2
2022-09-11 19:35:00,21601.52,21590.599597,21605.334298,2
2022-09-11 19:40:00,21603.17,21594.789731,21605.128174,2
2022-09-11 19:45:00,21592.01,21593.863154,21603.878824,2
2022-09-11 19:50:00,21614.71,21600.812103,21604.910365,2
2022-09-11 19:55:00,21618.13,21606.584735,21606.169378,1
2022-09-11 20:00:00,21656.21,21623.126490,21610.935151,1


In [144]:
def check():
    for symbol in symbols:
        if len(qry(symbol).position) > 1:
            #if qry(symbol).position[-1] and qry(symbol).position.diff()[-1]:
            if qry(symbol).position[-1] == 1 and qry(symbol).position[-1] - qry(symbol).position[-2] == -1:
                st.write(symbol+" LONG")
            elif qry(symbol).position[-1] == 2 and qry(symbol).position[-1] - qry(symbol).position[-2] == 1:
                st.write(symbol+" SHORT")
            

In [ ]:
st.button('Get EMA Cross', on_click=check())